# Hyperparameter Experiments

**Tujuan notebook ini:**
1. Test berbagai hyperparameter dengan epochs rendah
2. Bandingkan learning rates, batch sizes, dan architectures
3. Find optimal configuration sebelum full training
4. Validate fusion strategies

**⚠️ Jalankan notebook ini SETELAH validation notebook dan SEBELUM full training!**

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from sklearn.metrics import classification_report, accuracy_score

# Setup TensorFlow for AMD GPU (DirectML Plugin)
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")

# Setup DirectML Plugin for AMD GPU
try:
    # Check for GPU and DML devices
    gpu_devices = tf.config.list_physical_devices('GPU')
    dml_devices = tf.config.list_physical_devices('DML')
    
    total_devices = len(gpu_devices) + len(dml_devices)
    
    if total_devices > 0:
        print(f"\n🎮 AMD GPU DETECTED: {total_devices} device(s)")
        
        # Configure GPU memory growth
        for gpu in gpu_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU memory growth enabled")
        
        # Test GPU functionality
        with tf.device('/GPU:0'):
            a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
            b = tf.constant([[2.0, 1.0], [1.0, 2.0]])
            c = tf.matmul(a, b)
            print("✅ GPU computation test passed")
            
        gpu_available = True
        print("🚀 AMD GPU ready for training!")
        
    else:
        print("⚠️ No GPU detected. Using CPU mode.")
        gpu_available = False
        
except Exception as e:
    print(f"⚠️ GPU setup failed: {e}")
    print("💡 Falling back to CPU mode")
    gpu_available = False

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, Input, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

print(f"\n✅ Libraries imported successfully!")
print(f"🎮 GPU Status: {'Enabled' if gpu_available else 'Disabled (CPU only)'}")

TensorFlow version: 2.10.0

🎮 AMD GPU DETECTED: 2 device(s)
✅ GPU memory growth enabled
✅ GPU computation test passed
🚀 AMD GPU ready for training!

✅ Libraries imported successfully!
🎮 GPU Status: Enabled


## 1. 📁 Load Data

Load data yang sudah dipersiapkan untuk training.

In [4]:
# Base path untuk data
BASE_PATH = '../data/'

print("Loading data...")
try:
    # Load image data
    X_train_images = np.load(BASE_PATH + 'images/X_train_images.npy')
    X_val_images = np.load(BASE_PATH + 'images/X_val_images.npy')
    X_test_images = np.load(BASE_PATH + 'images/X_test_images.npy')
    
    # Load landmark data
    X_train_landmarks = np.load(BASE_PATH + 'landmarks/X_train_landmarks.npy')
    X_val_landmarks = np.load(BASE_PATH + 'landmarks/X_val_landmarks.npy')
    X_test_landmarks = np.load(BASE_PATH + 'landmarks/X_test_landmarks.npy')
    
    # Load labels
    y_train = np.load(BASE_PATH + 'images/y_train_images.npy')
    y_val = np.load(BASE_PATH + 'images/y_val_images.npy')
    y_test = np.load(BASE_PATH + 'images/y_test_images.npy')
    
    print(f"✅ Data loaded successfully!")
    print(f"Training samples: {X_train_images.shape[0]:,}")
    print(f"Validation samples: {X_val_images.shape[0]:,}")
    print(f"Test samples: {X_test_images.shape[0]:,}")
    print(f"Image shape: {X_train_images.shape[1:]}")
    print(f"Landmark features: {X_train_landmarks.shape[1]}")
    
except FileNotFoundError as e:
    print(f"❌ Error loading data: {e}")
    print("💡 Please check if data files exist in the correct path")
    print("💡 Run data preprocessing first if not done yet")

Loading data...
✅ Data loaded successfully!
Training samples: 3,123
Validation samples: 390
Test samples: 391
Image shape: (224, 224, 3)
Landmark features: 136


In [5]:
# Create label mapping
unique_labels = np.unique(y_train)
label_map = {label: i for i, label in enumerate(unique_labels)}
num_classes = len(unique_labels)
target_names = list(unique_labels)

print(f"Number of classes: {num_classes}")
print(f"Classes: {target_names}")
print(f"Label mapping: {label_map}")

# Convert labels to numerical
y_train_num = np.array([label_map[label] for label in y_train])
y_val_num = np.array([label_map[label] for label in y_val])
y_test_num = np.array([label_map[label] for label in y_test])

# Convert to one-hot encoding
y_train_onehot = to_categorical(y_train_num, num_classes)
y_val_onehot = to_categorical(y_val_num, num_classes)
y_test_onehot = to_categorical(y_test_num, num_classes)

print("✅ Label conversion completed")

Number of classes: 6
Classes: ['angry', 'disgusted', 'happy', 'neutral', 'sad', 'surprised']
Label mapping: {'angry': 0, 'disgusted': 1, 'happy': 2, 'neutral': 3, 'sad': 4, 'surprised': 5}
✅ Label conversion completed


## 2. 🔧 Define Model Architectures

Definisikan berbagai variasi model untuk testing.

In [6]:
def build_cnn_model(input_shape, num_classes, complexity='medium', dropout_rate=0.25):
    """Build CNN model with different complexity levels"""
    
    if complexity == 'simple':
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(128, activation='relu'),
            Dropout(dropout_rate),
            Dense(num_classes, activation='softmax')
        ])
    
    elif complexity == 'medium':
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            
            Conv2D(64, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            
            Conv2D(128, (3, 3), activation='relu'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            
            Flatten(),
            Dense(256, activation='relu'),
            Dropout(dropout_rate),
            Dense(128, activation='relu'),
            Dropout(dropout_rate),
            Dense(num_classes, activation='softmax')
        ])
    
    return model

def build_landmark_model(input_shape, num_classes, complexity='medium', dropout_rate=0.3):
    """Build landmark model with different complexity levels"""
    
    if complexity == 'simple':
        model = Sequential([
            Dense(128, activation='relu', input_shape=(input_shape,)),
            Dropout(dropout_rate),
            Dense(64, activation='relu'),
            Dropout(dropout_rate),
            Dense(num_classes, activation='softmax')
        ])
    
    elif complexity == 'medium':
        model = Sequential([
            Dense(128, activation='relu', input_shape=(input_shape,)),
            BatchNormalization(),
            Dropout(dropout_rate),
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(dropout_rate),
            Dense(128, activation='relu'),
            BatchNormalization(),
            Dropout(dropout_rate),
            Dense(num_classes, activation='softmax')
        ])
    
    return model

print("✅ Model architectures defined!")

✅ Model architectures defined!


## 3. 🧪 Setup Experiments

Setup experiment configurations dan data samples.

In [7]:
# Experiment configurations
experiment_configs = {
    'learning_rates': [0.001, 0.0001],
    'batch_sizes': [32, 16] if gpu_available else [16, 8],
    'complexities': ['simple', 'medium'],
    'dropout_rates': [0.25, 0.3]
}

# Adjust experiment size based on GPU availability
if gpu_available:
    experiment_size = min(2000, X_train_images.shape[0])
    val_size = min(500, X_val_images.shape[0])
else:
    experiment_size = min(1000, X_train_images.shape[0])
    val_size = min(200, X_val_images.shape[0])

# Create experiment datasets
X_train_exp = X_train_images[:experiment_size]
X_train_landmarks_exp = X_train_landmarks[:experiment_size]
y_train_exp = y_train_onehot[:experiment_size]

X_val_exp = X_val_images[:val_size]
X_val_landmarks_exp = X_val_landmarks[:val_size]
y_val_exp = y_val_onehot[:val_size]

print(f"🧪 EXPERIMENT SETUP:")
print(f"   Training samples: {experiment_size:,}")
print(f"   Validation samples: {val_size:,}")
print(f"   Device: {'GPU' if gpu_available else 'CPU'}")
print(f"   Configs to test: {len(experiment_configs['learning_rates']) * len(experiment_configs['batch_sizes']) * len(experiment_configs['complexities'])}")

experiment_results = []

🧪 EXPERIMENT SETUP:
   Training samples: 2,000
   Validation samples: 390
   Device: GPU
   Configs to test: 8


## 4. 🚀 Run CNN Experiments

Test berbagai konfigurasi CNN dengan epochs rendah.

In [8]:
def run_single_experiment(model_type, lr, batch_size, complexity, dropout_rate, epochs=5):
    """Run single experiment with given parameters"""
    
    print(f"Testing {model_type}: LR={lr}, BS={batch_size}, Complexity={complexity}")
    
    start_time = time.time()
    
    try:
        # Build model
        if model_type == 'cnn':
            model = build_cnn_model(X_train_exp.shape[1:], num_classes, complexity, dropout_rate)
            X_train_model = X_train_exp
            X_val_model = X_val_exp
        else:  # landmark
            model = build_landmark_model(X_train_landmarks_exp.shape[1], num_classes, complexity, dropout_rate)
            X_train_model = X_train_landmarks_exp
            X_val_model = X_val_landmarks_exp
        
        # Compile model
        model.compile(
            optimizer=Adam(learning_rate=lr),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # Train model
        history = model.fit(
            X_train_model, y_train_exp,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(X_val_model, y_val_exp),
            verbose=0
        )
        
        # Get results
        final_train_acc = history.history['accuracy'][-1]
        final_val_acc = history.history['val_accuracy'][-1]
        overfitting = final_train_acc - final_val_acc
        experiment_time = time.time() - start_time
        
        result = {
            'model_type': model_type,
            'learning_rate': lr,
            'batch_size': batch_size,
            'complexity': complexity,
            'dropout_rate': dropout_rate,
            'final_train_acc': final_train_acc,
            'final_val_acc': final_val_acc,
            'overfitting': overfitting,
            'experiment_time': experiment_time,
            'parameters': model.count_params(),
            'status': 'success'
        }
        
        print(f"   ✅ Val Acc: {final_val_acc:.4f}, Time: {experiment_time:.1f}s")
        
        # Clear memory
        del model
        tf.keras.backend.clear_session()
        
        return result
        
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        return {
            'model_type': model_type,
            'learning_rate': lr,
            'batch_size': batch_size,
            'complexity': complexity,
            'status': 'failed',
            'error': str(e)
        }

print("✅ Experiment function defined!")

✅ Experiment function defined!


In [ ]:
# Run CNN experiments
print("🖼️ RUNNING CNN EXPERIMENTS...")
print("=" * 40)

cnn_results = []
experiment_count = 0

for lr in experiment_configs['learning_rates']:
    for bs in experiment_configs['batch_sizes']:
        for complexity in experiment_configs['complexities']:
            experiment_count += 1
            dropout = 0.25 if complexity == 'simple' else 0.3
            epochs = 8 if gpu_available else 5
            
            result = run_single_experiment('cnn', lr, bs, complexity, dropout, epochs)
            cnn_results.append(result)
            experiment_results.append(result)

successful_cnn = len([r for r in cnn_results if r['status'] == 'success'])
print(f"\n✅ CNN experiments completed: {successful_cnn}/{len(cnn_results)} successful")

🖼️ RUNNING CNN EXPERIMENTS...
Testing cnn: LR=0.001, BS=32, Complexity=simple
   ✅ Val Acc: 0.8282, Time: 23.3s
Testing cnn: LR=0.001, BS=32, Complexity=medium


In [ ]:
# Run Landmark experiments
print("\n🎯 RUNNING LANDMARK EXPERIMENTS...")
print("=" * 40)

landmark_results = []

for lr in experiment_configs['learning_rates']:
    for bs in experiment_configs['batch_sizes']:
        for complexity in experiment_configs['complexities']:
            dropout = 0.3 if complexity == 'simple' else 0.4
            epochs = 10 if gpu_available else 6
            
            result = run_single_experiment('landmark', lr, bs, complexity, dropout, epochs)
            landmark_results.append(result)
            experiment_results.append(result)

successful_landmark = len([r for r in landmark_results if r['status'] == 'success'])
print(f"\n✅ Landmark experiments completed: {successful_landmark}/{len(landmark_results)} successful")

## 5. 📊 Analyze Results

Analisis hasil eksperimen untuk menemukan hyperparameter terbaik.

In [ ]:
# Analyze results
successful_results = [r for r in experiment_results if r['status'] == 'success']

if len(successful_results) > 0:
    df_results = pd.DataFrame(successful_results)
    
    print(f"📊 Analyzing {len(successful_results)} successful experiments...")
    
    # Best results per model type
    print("\n🏆 BEST RESULTS PER MODEL TYPE:")
    for model_type in df_results['model_type'].unique():
        model_results = df_results[df_results['model_type'] == model_type]
        best_idx = model_results['final_val_acc'].idxmax()
        best = model_results.loc[best_idx]
        
        print(f"\n{model_type.upper()}:")
        print(f"   Best Val Acc: {best['final_val_acc']:.4f}")
        print(f"   Learning Rate: {best['learning_rate']}")
        print(f"   Batch Size: {best['batch_size']}")
        print(f"   Complexity: {best['complexity']}")
        print(f"   Overfitting: {best['overfitting']:.4f}")
    
    # Top 3 overall
    print("\n🌟 TOP 3 OVERALL RESULTS:")
    top_3 = df_results.nlargest(3, 'final_val_acc')
    for idx, row in top_3.iterrows():
        print(f"{row['model_type']}: Val Acc {row['final_val_acc']:.4f}, LR {row['learning_rate']}, BS {row['batch_size']}")
    
else:
    print("❌ No successful experiments to analyze!")

In [ ]:
# Visualize results
if len(successful_results) > 0 and len(df_results) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Validation accuracy by learning rate
    df_results.boxplot(column='final_val_acc', by='learning_rate', ax=axes[0,0])
    axes[0,0].set_title('Validation Accuracy by Learning Rate')
    
    # Validation accuracy by batch size
    df_results.boxplot(column='final_val_acc', by='batch_size', ax=axes[0,1])
    axes[0,1].set_title('Validation Accuracy by Batch Size')
    
    # Overfitting by model type
    df_results.boxplot(column='overfitting', by='model_type', ax=axes[1,0])
    axes[1,0].set_title('Overfitting by Model Type')
    axes[1,0].axhline(y=0, color='r', linestyle='--', alpha=0.5)
    
    # Training time by complexity
    df_results.boxplot(column='experiment_time', by='complexity', ax=axes[1,1])
    axes[1,1].set_title('Training Time by Complexity')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("⚠️ No data to visualize")

## 6. 💾 Save Results & Recommendations

Simpan hasil experiment dan berikan rekomendasi untuk training penuh.

In [ ]:
# Save results and recommendations
if len(successful_results) > 0:
    # Create recommendations
    recommendations = {
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
        'experiment_summary': {
            'total_experiments': len(experiment_results),
            'successful_experiments': len(successful_results),
            'gpu_used': gpu_available
        },
        'best_configurations': {}
    }
    
    # Best config for each model type
    for model_type in df_results['model_type'].unique():
        model_results = df_results[df_results['model_type'] == model_type]
        best = model_results.loc[model_results['final_val_acc'].idxmax()]
        
        recommendations['best_configurations'][model_type] = {
            'learning_rate': best['learning_rate'],
            'batch_size': int(best['batch_size']),
            'complexity': best['complexity'],
            'dropout_rate': best['dropout_rate'],
            'expected_val_acc': best['final_val_acc']
        }
    
    # Save to file
    with open('experiment_results.pkl', 'wb') as f:
        pickle.dump({
            'detailed_results': experiment_results,
            'recommendations': recommendations
        }, f)
    
    print("💾 Results saved to 'experiment_results.pkl'")
    
    # Display final recommendations
    print("\n" + "=" * 50)
    print("🎯 FINAL RECOMMENDATIONS")
    print("=" * 50)
    
    for model_type, config in recommendations['best_configurations'].items():
        print(f"\n{model_type.upper()} MODEL:")
        print(f"   Learning Rate: {config['learning_rate']}")
        print(f"   Batch Size: {config['batch_size']}")
        print(f"   Complexity: {config['complexity']}")
        print(f"   Dropout Rate: {config['dropout_rate']}")
        print(f"   Expected Val Acc: {config['expected_val_acc']:.4f}")
    
    print("\n📋 NEXT STEPS:")
    print("   1. Update training scripts dengan parameter di atas")
    print("   2. Jalankan full training dengan early stopping")
    print("   3. Monitor training progress closely")
    print("   4. Test fusion strategies setelah individual models selesai")
    
    print("\n🚀 READY FOR FULL TRAINING!")
    
else:
    print("❌ No successful experiments to save")